In [ ]:
# import pandas as pd
# import torch
# from PIL import Image
# import requests
# from io import BytesIO
# from sentence_transformers import SentenceTransformer, util

# # Load dataset
# df = pd.read_csv("amazon_fashion_trimmed.csv")

# # Load CLIP model
# model = SentenceTransformer("clip-ViT-B-32")

# # Generate embeddings
# text_embeddings = model.encode(df["title"].tolist(), convert_to_tensor=True)

# image_embeddings = []
# for url in df["image_large"]:
#     try:
#         img = Image.open(BytesIO(requests.get(url).content)).convert("RGB")
#         image_embeddings.append(model.encode(img, convert_to_tensor=True))
#     except:
#         image_embeddings.append(torch.zeros(text_embeddings.shape[1]))  # fallback

# image_embeddings = torch.stack(image_embeddings)

# # Fusion (equal weights here, can be dynamic later)
# fused_embeddings = 0.5 * text_embeddings + 0.5 * image_embeddings
# fused_embeddings = torch.nn.functional.normalize(fused_embeddings, p=2, dim=1)

# # Recommendation function
# def recommend(idx, top_k=5):
#     query_emb = fused_embeddings[idx]
#     scores = util.cos_sim(query_emb, fused_embeddings)[0]
#     top_results = torch.topk(scores, k=top_k + 1)
    
#     print(f"Query: {df.iloc[idx]['title']}")
#     for score, i in zip(top_results[0][1:], top_results[1][1:]):  # skip itself
#         print(f"{df.iloc[i]['title']} (score: {score:.4f})")

# # Example
# recommend(0, top_k=5)


In [ ]:
# import os
# import pandas as pd
# import torch
# from PIL import Image
# import requests
# from io import BytesIO
# from sentence_transformers import SentenceTransformer, util
# from concurrent.futures import ThreadPoolExecutor
# import pickle

# # ===============================
# # CONFIG
# # ===============================
# DATA_FILE = "amazon_fashion_trimmed.csv"
# TEXT_EMB_CACHE = "text_embeddings.pt"
# IMAGE_EMB_CACHE = "image_embeddings.pt"
# MODEL_NAME = "clip-ViT-B-32"
# NUM_THREADS = 8  # Parallel image downloads
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# # ===============================
# # Load dataset
# # ===============================
# df = pd.read_csv(DATA_FILE)

# # ===============================
# # Load CLIP model
# # ===============================
# model = SentenceTransformer(MODEL_NAME, device=DEVICE)

# # ===============================
# # Encode text (cached)
# # ===============================
# # if os.path.exists(TEXT_EMB_CACHE):
# #     print("Loading cached text embeddings...")
# #     text_embeddings = torch.load(TEXT_EMB_CACHE, map_location=DEVICE)
# # else:
# #     print("Generating text embeddings...")
# #     text_embeddings = model.encode(df["title"].tolist(), convert_to_tensor=True, show_progress_bar=True)
# #     torch.save(text_embeddings, TEXT_EMB_CACHE)
# print("Generating text embeddings...")
# text_embeddings = model.encode(df["title"].tolist(), convert_to_tensor=True, show_progress_bar=True)

# # ===============================
# # Encode images in parallel (cached)
# # ===============================
# # def process_image(url):
# #     try:
# #         img = Image.open(BytesIO(requests.get(url, timeout=5).content)).convert("RGB")
# #         return model.encode(img, convert_to_tensor=True)
# #     except Exception as e:
# #         return torch.zeros(text_embeddings.shape[1])  # Fallback for failed images

# # if os.path.exists(IMAGE_EMB_CACHE):
# #     print("Loading cached image embeddings...")
# #     image_embeddings = torch.load(IMAGE_EMB_CACHE, map_location=DEVICE)
# # else:
# #     print("Downloading & encoding images in parallel...")
# #     with ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
# #         image_embeddings_list = list(executor.map(process_image, df["image_large"].tolist()))
# #     image_embeddings = torch.stack(image_embeddings_list)
# #     torch.save(image_embeddings, IMAGE_EMB_CACHE)

# def process_image(url):
#     try:
#         img = Image.open(BytesIO(requests.get(url, timeout=5).content)).convert("RGB")
#         return model.encode(img, convert_to_tensor=True)
#     except Exception:
#         return torch.zeros(text_embeddings.shape[1])  # Fallback for failed images

# print("Downloading & encoding images in parallel...")
# with ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
#     image_embeddings_list = list(executor.map(process_image, df["image_large"].tolist()))
# image_embeddings = torch.stack(image_embeddings_list)


# # ===============================
# # Fusion (equal weight for now)
# # ===============================
# fused_embeddings = 0.5 * text_embeddings + 0.5 * image_embeddings
# fused_embeddings = torch.nn.functional.normalize(fused_embeddings, p=2, dim=1)

# # ===============================
# # Recommendation function
# # ===============================
# def recommend(idx, top_k=5):
#     query_emb = fused_embeddings[idx]
#     scores = util.cos_sim(query_emb, fused_embeddings)[0]
#     top_results = torch.topk(scores, k=top_k + 1)

#     print(f"\nQuery: {df.iloc[idx]['title']}")
#     for score, i in zip(top_results[0][1:], top_results[1][1:]):  # skip itself
#         i = int(i)  # Convert tensor to int
#         print(f"{df.iloc[i]['title']} (score: {score.item():.4f})")

# # ===============================
# # Example usage
# # ===============================
# recommend(0, top_k=5)


In [ ]:
# from IPython.display import display

# def recommend(idx, top_k=5):
#     query_emb = fused_embeddings[idx]
#     scores = util.cos_sim(query_emb, fused_embeddings)[0]
#     top_results = torch.topk(scores, k=top_k + 1)

#     # Show the query item
#     print(f"\nQuery: {df.iloc[idx]['title']}")
#     try:
#         img = Image.open(BytesIO(requests.get(df.iloc[idx]['image_large']).content)).convert("RGB")
#         display(img)
#     except:
#         print("[Image not available]")

#     # Show recommendations
#     print("\nTop Recommendations:")
#     for score, i in zip(top_results[0][1:], top_results[1][1:]):  # skip itself
#         i = int(i)  # Convert tensor to int
#         print(f"{df.iloc[i]['title']} (score: {score.item():.4f})")
#         try:
#             img = Image.open(BytesIO(requests.get(df.iloc[i]['image_large']).content)).convert("RGB")
#             display(img)
#         except:
#             print("[Image not available]")


In [ ]:
import os
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

# ===============================
# CONFIG
# ===============================
DATA_FILE = "amazon_fashion_trimmed_10000.csv"
TEXT_EMB_CACHE = "text_embeddings.pt"
IMAGE_EMB_CACHE = "image_embeddings.pt"
MODEL_NAME = "clip-ViT-B-32"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ===============================
# Load dataset
# ===============================
df = pd.read_csv(DATA_FILE)

# ===============================
# Load CLIP model
# ===============================
model = SentenceTransformer(MODEL_NAME, device=DEVICE)

# ===============================
# Load cached text embeddings
# ===============================
if os.path.exists(TEXT_EMB_CACHE):
    print("Loading cached text embeddings...")
    text_embeddings = torch.load(TEXT_EMB_CACHE, map_location=DEVICE)
else:
    print("Generating text embeddings...")
    text_embeddings = model.encode(df["title"].tolist(), convert_to_tensor=True, show_progress_bar=True)
    torch.save(text_embeddings, TEXT_EMB_CACHE)

# ===============================
# Load cached image embeddings
# ===============================
if os.path.exists(IMAGE_EMB_CACHE):
    print("Loading cached image embeddings...")
    img_cache = torch.load(IMAGE_EMB_CACHE, map_location=DEVICE)
    
    # If the cache is a dict, convert to tensor in the same order as df
    if isinstance(img_cache, dict):
        image_embeddings = torch.stack([img_cache[path] for path in df["image_large"]])
    else:
        image_embeddings = img_cache
else:
    raise FileNotFoundError(f"{IMAGE_EMB_CACHE} not found! Please generate and save it first.")

# ===============================
# Fusion (equal weight for now)
# ===============================
fused_embeddings = 0.35 * text_embeddings + 0.65 * image_embeddings
fused_embeddings = torch.nn.functional.normalize(fused_embeddings, p=2, dim=1)

# ===============================
# Recommendation function
# ===============================
def recommend(idx, top_k=5):
    query_emb = fused_embeddings[idx]
    scores = util.cos_sim(query_emb, fused_embeddings)[0]
    top_results = torch.topk(scores, k=top_k + 1)

    # Show the query item
    print(f"\nQuery: {df.iloc[idx]['title']}")
    try:
        img = Image.open(BytesIO(requests.get(df.iloc[idx]['image_large']).content)).convert("RGB")
        display(img)
    except:
        print("[Image not available]")

    # Show recommendations
    print("\nTop Recommendations:")
    for score, i in zip(top_results[0][1:], top_results[1][1:]):  # skip itself
        i = int(i)  # Convert tensor to int
        print(f"{df.iloc[i]['title']} (score: {score.item():.4f})")
        try:
            img = Image.open(BytesIO(requests.get(df.iloc[i]['image_large']).content)).convert("RGB")
            display(img)
        except:
            print("[Image not available]")

# ===============================
# Example usage
# ===============================
# recommend(0, top_k=5)


In [ ]:
recommend(8682, top_k=5)